<a href="https://colab.research.google.com/github/serdaraydemr/imdb_recommendation/blob/main/%C4%B1mdb_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kullanıcıların filmlere verdiği puanlara göre oluşan veri seti ile Star Wars filmini beğenen kullanıcı için bir  öneri sistemi kurmaya çalışıcaz. İki adet veri setimiz var. Birinde filmler; diğerinde kullanıcıların oy verdiği filim isimleri ve puanları var. Star Wars filminin diğer filmlerle korelasyonuna bakıldı. Az oy almış filmler çıkması diye 250 adetten fazla oy almayan filmler öneriden çıkarıldı. df_movie adıyla içeri aktarılan veri setinin boyutu 
(1682,2) ; df_user adıyla kaydedilen veri setinin boyutu (100003, 4)'dır.

Sonuç : 250'den fazla kullanıcı tarafından oy almış filmler arasında Star Wars filmini izleyen birinin beğenebileceği en yüksek korelasyona sahip 3 film aşağıda verilmiştir :

1-Empire Strikes Back, The (1980), Korelasyon ->	0.74

2-Return of the Jedi (1983) Korelasyon, ->	0.67	

3-Raiders of the Lost Ark (1981) Korelasyon, -> 0.53

In [1]:
import os 
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt


In [5]:
df_movie = pd.read_csv("/content/gdrive/MyDrive/github/ımdb/movie_id_titles.csv")
column_names = ["user_id","item_id","rating","timestamp"]
df_user = pd.read_csv("/content/gdrive/MyDrive/github/ımdb/users.data",sep="\t",names=column_names)

In [6]:
df_user.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [13]:
df_user.shape

(100003, 4)

In [7]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100003 entries, 0 to 100002
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    100003 non-null  int64
 1   item_id    100003 non-null  int64
 2   rating     100003 non-null  int64
 3   timestamp  100003 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB


In [21]:
df_user.describe()

,user_id,item_id,rating,timestamp
count,100003.000000,100003.000000,100003.000000,1.000030e+05
mean,462.470876,425.520914,3.529864,8.835288e+08
std,266.622454,330.797791,1.125704,5.343791e+06
min,0.000000,1.000000,1.000000,8.747247e+08
25%,254.000000,175.000000,3.000000,8.794487e+08
50%,447.000000,322.000000,4.000000,8.828269e+08
75%,682.000000,631.000000,4.000000,8.882600e+08
max,943.000000,1682.000000,5.000000,8.932866e+08


In [14]:
df_user.isnull().sum()

user_id      0
item_id      0
rating       0
timestamp    0
dtype: int64

In [16]:
df_user.groupby("rating").size()

rating
1     6111
2    11370
3    27145
4    34174
5    21203
dtype: int64

In [17]:
df_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   item_id  1682 non-null   int64 
 1   title    1682 non-null   object
dtypes: int64(1), object(1)
memory usage: 26.4+ KB


In [18]:
df_movie.isnull().sum()

item_id    0
title      0
dtype: int64

In [22]:
df_movie.shape

(1682, 2)

In [23]:
df = pd.merge(df_user,df_movie, on ="item_id") # sql join işlemi.

In [28]:
df.head(25)

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)
5,274,50,5,878944679,Star Wars (1977)
6,227,50,4,879035347,Star Wars (1977)
7,99,50,5,885679998,Star Wars (1977)
8,305,50,5,886321799,Star Wars (1977)
9,108,50,4,879879739,Star Wars (1977)


Recommendation Sistemi Kuruyoruz.

In [29]:
new_df = df.pivot_table(index="user_id",columns="title",values="rating")
new_df.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# Star Wars filminin ratinglerine bakalım 
starwars = new_df['Star Wars (1977)']
starwars.head()

user_id
0    5.0
1    5.0
2    5.0
3    NaN
4    5.0
Name: Star Wars (1977), dtype: float64

In [37]:
print("Star Wars (1997) filminin Rating ortalaması : {}".format(starwars.mean()))

Star Wars (1997) filminin Rating ortalaması : 4.359589041095891


In [38]:
# Amacımız Star Wars filmine benzer film önerileri yapmak.
# corwith() metodunu kullanarak Star Wars filmi ile korelasyon hesaplayalım

similar_to_starwars = new_df.corrwith(starwars)
similar_to_starwars

/usr/local/lib/python3.9/dist-packages/numpy/lib/function_base.py:2821: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.9/dist-packages/numpy/lib/function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


title
'Til There Was You (1997)                0.872872
1-900 (1994)                            -0.645497
101 Dalmatians (1996)                    0.211132
12 Angry Men (1957)                      0.184289
187 (1997)                               0.027398
                                           ...   
Young Guns II (1990)                     0.228615
Young Poisoner's Handbook, The (1995)   -0.007374
Zeus and Roxanne (1997)                  0.818182
unknown                                  0.723123
Á köldum klaka (Cold Fever) (1994)            NaN
Length: 1664, dtype: float64

In [42]:
type(similar_to_starwars)

pandas.core.series.Series

In [43]:
# dataframe çevirelim 
corr_starwars = pd.DataFrame(similar_to_starwars, columns=['Correlation'])
corr_starwars.dropna(inplace=True)
corr_starwars.head()

,Correlation
title,
'Til There Was You (1997),0.872872
1-900 (1994),-0.645497
101 Dalmatians (1996),0.211132
12 Angry Men (1957),0.184289
187 (1997),0.027398


In [48]:
corr_starwars.sort_values('Correlation',ascending=False).head(10)

,Correlation
title,
Hollow Reed (1996),1.0
Commandments (1997),1.0
Cosi (1996),1.0
No Escape (1994),1.0
Stripes (1981),1.0
Star Wars (1977),1.0
Man of the Year (1995),1.0
"Beans of Egypt, Maine, The (1994)",1.0
"Old Lady Who Walked in the Sea, The (Vieille qui marchait dans la mer, La) (1991)",1.0


In [49]:
# Alakasız sonuçlar çıktı. Çıkan filmlerin çok az oy alması nedeniyle sanki alakalı gibi gözüküyor.
# Bu yüzden 250'den az oyu olan filmleri eleyelim

df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


In [59]:
# timestamp	işimize yaramıcak silebilriiz.
df.drop(['timestamp'],axis=1)

,user_id,item_id,rating,title
0,0,50,5,Star Wars (1977)
1,290,50,5,Star Wars (1977)
2,79,50,4,Star Wars (1977)
3,2,50,5,Star Wars (1977)
4,8,50,5,Star Wars (1977)
...,...,...,...,...
99998,840,1674,4,Mamma Roma (1962)
99999,655,1640,3,"Eighth Day, The (1996)"
100000,655,1637,3,Girls Town (1996)
100001,655,1630,3,"Silence of the Palace, The (Saimt el Qusur) (1..."


In [60]:
# her filmin ortalama rating değerlerine bakalım 
ratings = pd.DataFrame(df.groupby('title')['rating'].mean())
# Büyükten küçüğe sıralayalım
ratings.sort_values('rating',ascending=False).head()

,rating
title,
They Made Me a Criminal (1939),5.0
Marlene Dietrich: Shadow and Light (1996),5.0
"Saint of Fort Washington, The (1993)",5.0
Someone Else's America (1995),5.0
Star Kid (1997),5.0


In [61]:
# Şimdi her filmin aldığı oy sayısnı bulalım 
ratings['rating_oy_sayisi'] = pd.DataFrame(df.groupby('title')['rating'].count())
ratings.head()

,rating,rating_oy_sayisi
title,,
'Til There Was You (1997),2.333333,9
1-900 (1994),2.600000,5
101 Dalmatians (1996),2.908257,109
12 Angry Men (1957),4.344000,125
187 (1997),3.024390,41


In [63]:
# oy sayısına göre büyükten küçüğe sıralayalım 
ratings.sort_values('rating_oy_sayisi',ascending=False).head(15)

,rating,rating_oy_sayisi
title,,
Star Wars (1977),4.359589,584
Contact (1997),3.803536,509
Fargo (1996),4.155512,508
Return of the Jedi (1983),4.007890,507
Liar Liar (1997),3.156701,485
"English Patient, The (1996)",3.656965,481
Scream (1996),3.441423,478
Toy Story (1995),3.878319,452
Air Force One (1997),3.631090,431


In [64]:
corr_starwars.head()

,Correlation
title,
'Til There Was You (1997),0.872872
1-900 (1994),-0.645497
101 Dalmatians (1996),0.211132
12 Angry Men (1957),0.184289
187 (1997),0.027398


In [65]:
corr_starwars = corr_starwars.join(ratings['rating_oy_sayisi'])
corr_starwars.head()

,Correlation,rating_oy_sayisi
title,,
'Til There Was You (1997),0.872872,9
1-900 (1994),-0.645497,5
101 Dalmatians (1996),0.211132,109
12 Angry Men (1957),0.184289,125
187 (1997),0.027398,41


In [67]:
corr_starwars[corr_starwars['rating_oy_sayisi']>250].sort_values('Correlation',ascending=False).head(15)

,Correlation,rating_oy_sayisi
title,,
Star Wars (1977),1.000000,584
"Empire Strikes Back, The (1980)",0.748353,368
Return of the Jedi (1983),0.672556,507
Raiders of the Lost Ark (1981),0.536117,420
Indiana Jones and the Last Crusade (1989),0.350107,331
L.A. Confidential (1997),0.319065,297
E.T. the Extra-Terrestrial (1982),0.303619,300
Back to the Future (1985),0.274839,350
Jaws (1975),0.265459,280
